<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import time
import threading
import random

app = dash.Dash(__name__)
server = app.server

# 步驟清單
step_names = [
    "撈取資料",
    "確認與篩選資料",
    "資料預處理",
    "進行 Rule Check",
    "繪圖",
    "上拋資料"
]

# 儲存進度資訊
progress = {
    'current_index': -1,
    'step_times': [None] * len(step_names),
    'step_status': ['pending'] * len(step_names),  # pending / running / success / error
    'error': False
}

# 執行報告產出流程
def run_report_process():
    for i in range(len(step_names)):
        if progress['error']:
            break

        progress['current_index'] = i
        progress['step_status'][i] = 'running'
        step_start = time.time()

        time.sleep(1)  # 模擬耗時操作

        if i == 3 and random.random() < 0.1:  # 模擬第4步失敗
            progress['step_status'][i] = 'error'
            progress['error'] = True
            break

        step_end = time.time()
        progress['step_times'][i] = round(step_end - step_start, 2)
        progress['step_status'][i] = 'success'


app.layout = html.Div([
    html.H2("報告產出流程 - 步驟追蹤"),
    html.Button("開始產出報告", id="run-btn", n_clicks=0),
    html.Div(id="step-list", style={"marginTop": "20px", "fontSize": "16px"}),
    html.Div(id="progress-label", style={"marginTop": "20px", "fontSize": "16px"}),
    html.Div(id="to-comment-container", style={"marginTop": "30px"}),
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=True)
])


@app.callback(
    Output("interval", "disabled"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_report(n_clicks):
    # 初始化進度
    progress['current_index'] = -1
    progress['step_times'] = [None] * len(step_names)
    progress['step_status'] = ['pending'] * len(step_names)
    progress['error'] = False

    thread = threading.Thread(target=run_report_process)
    thread.start()
    return False


@app.callback(
    Output("step-list", "children"),
    Output("progress-label", "children"),
    Output("to-comment-container", "children"),
    Input("interval", "n_intervals"),
    prevent_initial_call=True
)
def update_ui(n):
    current = progress['current_index']
    step_status = progress['step_status']
    step_times = progress['step_times']
    steps_total = len(step_names)

    # 產出步驟文字列表
    step_elements = []
    for i, step in enumerate(step_names):
        status = step_status[i]

        if status == 'pending':
            color = 'gray'
            prefix = "□"
        elif status == 'running':
            color = 'orange'
            prefix = "▶"
        elif status == 'success':
            color = 'green'
            prefix = "✔"
        elif status == 'error':
            color = 'red'
            prefix = "❌"
        else:
            color = 'gray'
            prefix = "□"

        time_spent = f" ({step_times[i]}秒)" if step_times[i] is not None else ""
        step_elements.append(html.Div(f"{prefix} {step}{time_spent}", style={"color": color, "marginBottom": "5px"}))

    # 狀態說明文字
    if progress['error']:
        label = f"流程中斷：{step_names[current]} 發生錯誤"
        comment_button = ""
    elif current >= steps_total - 1 and all(s == 'success' for s in step_status):
        label = "✅ 完成所有步驟！"
        comment_button = html.Button("To Comment", id="to-comment", style={"fontSize": "16px"})
    else:
        label = f"目前進度：{current + 1}/{steps_total}"
        comment_button = ""

    return step_elements, label, comment_button


if __name__ == "__main__":
    app.run(debug=True)
